# <div align="center">Detail understanding of SMPL paper</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


### <div align="center">Shape blend shapes</div>

The body shapes of different people are represented by a linear function $B_S$


<div align="center">$B_S(\beta; \mathcal{S}) = \displaystyle\sum_{n=1}^{|\beta|} \beta_n S_n$</div>

where

* $\beta = [\beta_1, ..., \beta_{|\beta|}]$, $|\beta|$ is the number of linear shape coefficients, and

* $S_n \in \mathcal{R}^{3N}$ represent orthonormal principal components of shape displacements.

Let $\mathcal{S} = [S_1,... , S_{|\beta|}] \in \mathcal{R}^{3N x |\beta|}$ be the matrix of all such shape displacements. Then the linear function $B_S(\beta; \mathcal{S})$ is fully defined by the matrix $\mathcal{S}$, which is learned from registered training meshes. Notationally, the values to the right of a semicolon represent learned parameters, while those on the left are parameters set by an animator. For notational convenience, we often omit the learned parameters when they are not explicitly being optimized in training.

### <div align="center">Pose blend shapes</div>

We denote as $R : \mathcal{R}^{\theta} \to \mathcal{R}^{9K}$ a function that maps a pose vector $\theta$ to a vector of concatenated part relative rotation matrices, $exp(\theta)$. Given that our rig has 23 joints, $R(\theta)$ is a vector of length (23 * 9 = 207). Elements of $R(\theta)$ are functions of sines and cosines of joint angles and therefore $R(\theta)$ is
non-linear with $\theta$.

Our formulation differs from previous work in that we define the effect of the pose blend shapes to be linear in $R^{*}(\theta) = R(\theta) - R(\theta^{*})$ , where $\theta^{*}$ denotes the rest pose. Let $R_n(\theta)$ denote the n-th element of $R(\theta)$, then the vertex deviations from the rest template are

<div align="center">$B_P(\theta; \mathcal{P}) = \displaystyle \sum_{n=1}^{9K} (R_n(\theta) - R_n(\theta^*))P_n$</div>

where 

* the blend shapes, $P_n \in \mathcal{R}^{3N}$, are again vectors of vertex displacements.

* $\mathcal{P} = [P_1, ..., P_{9K}] \in \mathcal{R}^{3N x 9K}$, is a matrix of all 207 pose blend shapes. In this way, the pose blend shape function $B_P(\theta; \mathcal{P})$ is fully defined by the matrix $\mathcal{P}$.

Note that subtracting the rest pose rotation vector, $R(\theta^*)$, guarantees that the contribution of the pose blend shapes is zero in the rest pose, which is important for animation.

### <div align="center">Joint locations</div>

Different body shapes have different joint locations. Each joint is represented by its 3D location in the rest pose. We define the joints as a function of the body shape, $\beta$

<div align="center">$J(\beta; \mathcal{J}, \bar{T}, \mathcal{S}) = \mathcal{J}(\bar{T} + B_S(\beta; \mathcal{S}))$</div>

where

* $\mathcal{J}$ is a matrix that transforms rest vertices into rest joints.

We learn the ***regression matrix***, $\mathcal{J}$ , from examples of different people in many poses, as part of our overall model learning. This matrix models which mesh vertices are important and how to combine them to estimate the joint locations.



### <div align="center">SMPL model</div>

We can now specify the full set of model parameters of the SMPL model as $\mathcal{\Phi} = \{ \bar{T}, \mathcal{W}, \mathcal{S}, \mathcal{J}, \mathcal{P} \}$. We describe how to learn these in next sections. Once learned they are held fixed and new body shapes and poses are created and animated by varying $\beta$ and $\theta$ respectively.

The SMPL model is finally defined as

<div align="center">$M(\beta, \theta; \mathcal{\Phi}) = W (T_P(\beta, \theta; \bar{T}, \mathcal{S}, \mathcal{P}), J(\beta; \mathcal{J}, \bar{T}, \mathcal{S}), \theta, \mathcal{W})$</div>

and hence each vertex is transformed as

<div align="center">$t_i' = \displaystyle{\sum_{k=1}^{K}} w_{k, i} G_K'(\theta, J(\beta; \mathcal{J}, \bar{T}, \mathcal{S})) t_{P, i}(\beta, \theta; \bar{T}, \mathcal{S}, \mathcal{P})$</div>

where 
* $t_{P, i}(\beta, \theta; \bar{T}, \mathcal{S}, \mathcal{P}) = \bar{t_i} + \displaystyle{\sum_{m=1}^{|\beta|}} \beta_m s_{m, i} + \displaystyle{\sum_{n=1}^{9K}}(R_n(\theta) - R_n(\theta^*))p_{n, i}$ represents the vertex $i$ after applying the blend shapes and 

where

* $s_{m, i}, p_{n, i} \in \mathcal{R}^3$ are the elements of the shape and pose blend shapes corresponding to template vertex $\bar{t_i}$.